In [1]:
import math
from math import pi as PI
import numpy as np
from numpy.linalg import norm
from numpy import sqrt
from matplotlib import animation, pyplot as plt
from helper import d_beta, d_beta_numeric, beta, simple_gaussian, broken_sigmoid, min_sep,\
    collision_trajectory, v2sp, sp2v, sp2a, rotate, theta, d_theta, d_theta_numeric, beta, d_beta, d_beta_numeric,\
    psi, d_psi, d_psi_numeric, phi, av2dsdp
from simulator import Simulation, Agent
from models import Model
import pandas as pd
%matplotlib qt


In [3]:
'''Simulator debug'''
# p0 = np.array(sim.p[0])
# p1 = np.array(sim.p[1])

# plt.scatter(p0[:,0], p0[:,1], s=1)
# plt.scatter(p1[:,0], p1[:,1], s=1)
# len(sim.t)
sim.plot_speeds()



In [3]:
'''Simulation'''
# Agent(id, goal_id, w, init_state, p_spd, model_names, model_args, constant, ref=[0, 1])
# -------------Models-------------
# {'name': 'mass_spring_approach', 'p_spd': 1.3, 't_relax': 3, 'b': 3.25, 'k_g': 7.5, 'c_1': 0.4, 'c_2': 0.4}
# {'name': 'optical_ratio_avoid', 'k_s': 0.1, 'k_h': 0.05, 'c': 1}
# {'name': 'perpendicular_acceleration_avoid', 'k': 2, 'c': 1}
# ------------Scenarios-----------
# 0 goal [{'p':[0, 5], 'v':[0, 0]}, {'p':[0, -5], 'v':[0, 1]}]
# 45 interception [{'p':[0, 5], 'v':[0, 0]}, {'p':[0, -5], 'v':[0, 1]}, {'p':[-5, 0], 'v':[1, 0]}]
# front interception [{'p':[0, 5], 'v':[0, 0]}, {'p':[0, -5], 'v':[0, 1]}, {'p':[3.5, 3.5], 's': 0.9, 'phi': -2.355}]
# back interception [{'p':[0, 5], 'v':[0, 0]}, {'p':[0, -5], 'v':[0, 1]}, {'p':[-3.5, -3.5], 's': 1.2, 'phi': 0.7}]
# head-on interception [{'p':[0, 5], 'v':[0, 0]}, {'p':[0, -5], 'v':[0, 1]}, {'p':[0, 5], 'v': [0, -1]}]

%matplotlib qt
models = [{'name': 'mass_spring_approach', 'p_spd': 1.3, 't_relax': 3, 'b': 3.25, 'k_g': 7.5, 'c_1': 0.4, 'c_2': 0.4},
         {'name': 'perpendicular_acceleration_avoid', 'k': 3, 'c': 1}]
states = [{'p':[0, 5], 'v':[0, 0]}, {'p':[0, -5], 'v':[0, 1]}, {'p':[5, 0], 'v':[-1, 0]}]
agents = []
# Goal
agents.append(Agent(0, constant=True))
# Agent 1
agents.append(Agent(1, goal_id=0, w=0.5, p_spd=1.3, models=models))
# Agent 2
agents.append(Agent(2, w=0.5, constant=True))

for state, agent in zip(states, agents):
    agent.set_init_state(state)
     
sim = Simulation(agents, 100)
sim.simulate(8)
sim.play(interval=None, save=False)
# sim.plot_speeds()

(<Figure size 1400x700 with 2 Axes>,
 <matplotlib.animation.FuncAnimation at 0x29c21bf89e8>)

In [3]:
# sim.play(interval=None, save=False)
# sim.a[1]
trajs = np.stack((p0, p1))
for traj in trajs:
    plt.plot([p[0] for p in traj], [p[1] for p in traj])

In [24]:
'''Convert a to d_s and d_phi'''
Hz = 100
t = 100
dt = 1 / Hz
#-------av-------
x = [0]
y = [0]
v = [0, 1]
a = [1, 0]
#-------sp-------
s, phi = v2sp(v)
v2 = v[:]
x2 = x[:]
y2 = y[:]
s = norm(v2)
d_s, d_phi = av2dsdp(v, a)

for i in range(t*Hz):
    #-------av-------
    x.append(x[-1] + v[0] * dt)
    y.append(y[-1] + v[1] * dt)
    v[0] += a[0] * dt
    v[1] += a[1] * dt
    #-------sp-------
    x2.append(x2[-1] + v2[0] * dt)
    y2.append(y2[-1] + v2[1] * dt)
    d_s, d_phi = av2dsdp(v2, a)
    s += d_s * dt
    phi += d_phi * dt
    v2 = sp2v(s, phi)
    
plt.figure(1)
plt.title('Overlap two trajectories')
plt.plot(x, y)
plt.plot(x2, y2)
plt.figure(2)
plt.title('Difference between two trajectories')
plt.plot([i - j for i, j in zip(x, x2)])
plt.plot([i - j for i, j in zip(y, y2)])
plt.legend(['difference on x', 'difference on y'])

In [25]:
'''Convert d_s d_phi to a'''
Hz = 100
t = 100
dt = 1 / Hz
#-----sp-------
dd_phi = 0.1
d_phi = 0
phi = 0
d_s = 0.1
s = 0
x = [0]
y = [0]
#-----av-------
a = sp2a(s, d_s, phi, d_phi)
v2 = sp2v(s, phi)
x2 = [0]
y2 = [0]

for i in range(t*Hz):
    #-----sp-------
    v = sp2v(s, phi)
    x.append(x[-1] + v[0] * dt)
    y.append(y[-1] + v[1] * dt)
    phi += d_phi * dt
    d_phi += dd_phi * dt
    s += d_s * dt
    #-----av-------
    x2.append(x2[-1] + v2[0] * dt)
    y2.append(y2[-1] + v2[1] * dt)
    v2[0] += a[0] * dt
    v2[1] += a[1] * dt
    a = sp2a(s, d_s, phi, d_phi)
    
plt.figure(1)
plt.title('Overlap two trajectories')
plt.plot(x, y)
plt.plot(x2, y2)
plt.figure(2)
plt.title('Difference between two trajectories')
plt.plot([i - j for i, j in zip(x, x2)])
plt.plot([i - j for i, j in zip(y, y2)])
plt.legend(['difference on x', 'difference on y'])

In [3]:
'''Debug angle helper functions'''
Hz = 100
w = 2
ref = [1, 0]
p0, p1, v0, v1 = collision_trajectory(45, 'f', spd1=2, w=w, r_min=w, Hz=Hz, animate=False, save=False)



d_phi = np.zeros(len(p0))
d_betas1 = d_beta_numeric(p0, p1, v0, Hz)
d_betas2 = d_beta(p0, p1, v0, v1, d_phi)

plt.plot(d_betas1)
plt.plot(d_betas2)


# Data output to file
# data = np.concatenate((p0, p1, v0, v1, a0, np.expand_dims(d_betas1, axis=1), np.expand_dims(d_betas2, axis=1)), axis=1)

# df = pd.DataFrame(data=data, columns=['p0x', 'p0y', 'p1x', 'p1y', 'v0x', 'v0y', 'v1x', 'v1y', 'a0x', 'a0y', 'd_beta_numeric', 'd_beta_analytic'])
# df.to_csv("d_beta.csv", index=False)


In [3]:
'''Debug a,v,s,p helper functions'''
a = [[0, 1], [1, 0], [1, 1]]
v = [[1, 0], [1, 0], [1, 0]]
av2dsdp(a, v)

array([[ 0.000000e+00, -1.000000e+00],
       [ 1.000000e+00,  6.123234e-17],
       [ 1.000000e+00, -1.000000e+00]])

In [5]:
'''Modeling ideas'''
Hz = 100
w = 0.5
traj0, traj1, v0, v1 = collision_trajectory(20, 'f', spd1=1.3, w=w, r_min=w, Hz=Hz, animate=True)
a0 = np.zeros_like(traj0)
thetas = theta(traj0, traj1, w)
d_betas = np.absolute(d_beta_numeric(traj0, traj1, v0, Hz=Hz))
d_thetas = d_theta(traj0, traj1, v0, v1, 2)
RRE = d_thetas / thetas
end = np.argmax(np.absolute(d_betas))

plt.figure()
# Rate of bearing change and rate of expansion
plt.plot(d_betas, label='d_betas numerical')
plt.plot(RRE, label='RRE')
plt.legend()

plt.figure()
# Critical rate of bearing change as a function of (relative) rate of expansion
c = 10
ratio = (RRE[:end] + c) / (d_betas[:end] + c) - 1
plt.plot(ratio)



In [17]:
# Plot rate of bearing change as a function of (relative) rate of expansion at different angle of interception
Hz = 100
for i in range(10, 80, 1):
    traj0, traj1, v0, v1 = collision_trajectory(i, 'f', Hz=Hz, animate=False)
    d_beta = rate_of_bearing(traj0, traj1, v0, Hz=Hz)
    d_theta = rate_of_expansion(traj0, traj1, v0, v1, 2, relative=False)
    end = np.argmax(d_beta)
#     plt.scatter(np.absolute(d_beta[: end]), d_theta[: end], marker='.', s=1)
    plt.scatter(range(0, len(d_beta[: end]/ d_theta[: end])), d_beta[: end]/ d_theta[: end], marker='.', s=1)
    

In [15]:
# Plot rate of bearing change as a function of (relative) rate of expansion at different speed
Hz = 100
for i in range(8, 16, 1):
    i = i / 10.0
    traj0, traj1, v0, v1 = collision_trajectory(45, 'f', spd1=i, Hz=Hz, animate=False)
    d_beta = rate_of_bearing(traj0, traj1, v0, Hz=Hz)
    d_theta = rate_of_expansion(traj0, traj1, v0, v1, 2, relative=False)
    end = np.argmax(d_beta)
    plt.scatter(range(0, len(d_beta[: end]/ d_theta[: end])), d_beta[: end]/ d_theta[: end], marker='.', s=1)

In [18]:
# Plot rate of bearing change as a function of (relative) rate of expansion with different width
Hz = 100
for i in range(2, 20, 1):
    i = i / 10.0
    traj0, traj1, v0, v1 = collision_trajectory(45, 'f', w=i, Hz=Hz, animate=False)
    d_beta = rate_of_bearing(traj0, traj1, v0, Hz=Hz)
    d_theta = rate_of_expansion(traj0, traj1, v0, v1, 2, relative=False)
    end = np.argmax(d_beta)
    plt.scatter(range(0, len(d_beta[: end]/ d_theta[: end])), d_beta[: end]/ d_theta[: end], marker='.', s=1)